# Collecting Data

This course is ultimately about data.

1. How do we get data?
    * How can we identify the data we need to make decisions?
    * How do we design experiments in order to acquire that data?
2. How do we validate data?
    * How do we know the data we've collected is correct?
    * How do we know we've collected the correct data?
3. How do we make inferences using data?
    * Once we have data, how do we estimate the uncertainty of the data?
    * If we use uncertain data to make inferences, how uncertain are those inferences?

Of course these questions make sense in any situation where we're dealing with empirical data, in this class we'll be working in the context of learning how to design experiements and collecting data using microcontrollers and electronic circuits.

The goal this week is to use a microcontroller to measure some of the characteristics of a circuit.

The circuit we'll be using is shown in Figure 1, below.

|Figure 1|
|:-|
|![Fig. 1: This Week's Circuit](images/divider.png)|
|This Week's Circuit|

Your instructor will provide your group with a microcontroller and a PCB with this circuit already constructed. You'll be using [CircuitPython](https://circuitpython.org/) to drive the `pin-17` terminal (labelled `Vdac` in the schematic) and collect data from the `pin-15` and `pin-10` terminals (labelled `adc1` and `adc2` in the schematic).

You'll notice that there's a capacitor in the circuit. For the first experiment, we'll drive the circuit very slowly, so that the capacitor plays no role. Later, we'll consider the transient behavior of the circuit and you'll see that the capacitor can have a dramatic effect.

# CircuitPython

[CircuitPython](https://circuitpython.org/) is a version of python that runs on small micro-controllers. It has many of the features of the python language you probably learned before, but it includes libraries that enable one to control and measure voltages on the input and output pins of the micro-controller. To program the microcontroller you'll need to take several steps.

1. Install a microcontroller aware editor on your laptop. I suggest [Thonny](https://thonny.org/). There are others, but thonny is the simplest to install and use.
2. Copy/Paste the "starter code" below into thonny and save "starter.py" to the device.
3. Run the starter code, watch the output window to see the data being collected.
4. Reset the device to see the updated output file.

## 1 Thonny

You can get Thonny [here](https://thonny.org/).

Once you have it installed you can connect the microcontroller and see the port listed as shown in Figure 2.

|Figure 2|
|:-|
|![Fig. 2: Thonny connected to ESP-S2](images/thonny.png)|
|Thonny connected.|



Starter code:

```
import board # information about the physical microcontroller
import analogio as aio # analog input and output library
import time # standard python time library

ADCMAX = 2**16-1 # maximum input/out from ADC or to DAC

adc1 = aio.AnalogIn(board.IO15)  # to read voltage from pin 15
adc2 = aio.AnalogIn(board.IO10)  # to read voltage from pin 10
vfactor = adc1.reference_voltage/ADCMAX # get conversion factor voltage to pin

vdac = aio.AnalogOut(board.IO17) # to control voltage on pin 17

f = open('starter.csv','w')
header = "j,vdac,v1,v2,time"


vdac.value = 0 # set output voltage to zero
print("sleeping.... let C fully discharge")
print("3")
time.sleep(1)
print("2")
time.sleep(1)
print("1")
time.sleep(1)
print("0 --- go!")

f.write(header + "\n")
print(header)

t0 = time.monotonic_ns()

for i in range(0,255,2): # steps of 2 to save time
    time.sleep(0.2) # let the cap charge....
    t = time.monotonic_ns()-t0
    vout = int((i/255)*ADCMAX)
    vdac.value = vout
    sval = f"{i},{vout*vfactor},{adc1.value*vfactor},{adc2.value*vfactor},{t/1e9}"
    f.write(f"{sval}\n")
    print(sval)

vdac.value = 0 # let the LED rest...
f.close()
```